In [1]:
from dotenv import dotenv_values
from openai import OpenAI
from pathlib import Path
import json

In [2]:
env = dotenv_values(Path(".") / ".." / ".env")

In [3]:
openai_client = OpenAI(api_key=env["OPENAI_API_KEY"])

In [4]:
PROJECTS_PATH = Path(".") / ".." / "docs" / "projects"

In [6]:
PAGE_PATH =  PROJECTS_PATH / "eda_wroclaw_halfmarathon" / "eda_as_presentation"
PDF_NAME = "eda.pdf"
PDF_PATH = PAGE_PATH / PDF_NAME

with open(PAGE_PATH / "notes.txt", "r") as f:
    NOTES = f.read()

AUTHOR_NAME = "Dariusz Samól"
AUTHOR_LINK = "/od-zera-do-ai-portfolio/uczestnicy/dariusz_samol"

## Opis projektu

In [7]:
def add_title_and_description(text):

    prompt = f"""
Wygeneruj tytuł i opis do strony prezentującej projekt.
Opis ma być wstępem przygotowującym czytelnika do zapoznania się z samym projektem.

{text}
    Zwróć obiekt JSON z kluczami "title" i "description".
    """

    response = openai_client.chat.completions.create(
        messages=[
            {"role": "user", "content": prompt},
        ],
        model="gpt-4o",
        temperature=0,
    )

    result = response.choices[0].message.content.replace("```json", "").replace("```", "").strip()

    return json.loads(result)

In [8]:
title_and_desc = add_title_and_description(NOTES)

## Wygeneruj index.md

In [9]:
def render_content(title, description, author_name, author_link, pdf_link, pdf_height):

    return f"""
# {title}

{description}

**Autor**: [{author_name}]({author_link})

<a href="{pdf_link}" class="md-button md-button--primary">Pobierz PDF</a>
<iframe
    id="content"
    src="{pdf_link}"
    width="100%"
    height="{pdf_height}px"
    style="border:1px solid black"
></iframe>

"""

In [10]:
with open(PAGE_PATH / "index.md", "w") as f:
    f.write(render_content(
        title=title_and_desc["title"],
        description=title_and_desc["description"],
        author_name=AUTHOR_NAME,
        author_link=AUTHOR_LINK,
        pdf_link=PDF_NAME,
        pdf_height=500
    ))